In [1]:
!pip install transformers
!pip install -q transformers

In [2]:
from nltk.corpus import stopwords
import os
import nltk
import sklearn
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [3]:
df = pd.read_csv("/content/BC7-LitCovid-Train.csv")
df2 = pd.read_csv("/content/BC7-LitCovid-Dev (1).csv")

print(df.head)

<bound method NDFrame.head of            pmid  ...                 label
0      32519164  ...   Treatment;Mechanism
1      32691006  ...  Treatment;Prevention
2      32858315  ...           Case Report
3      32985329  ...            Prevention
4      32812051  ...             Treatment
...         ...  ...                   ...
24955  32508388  ...            Prevention
24956  32815519  ...             Diagnosis
24957  32904987  ...   Treatment;Diagnosis
24958  32892181  ...            Prevention
24959  32389144  ...            Prevention

[24960 rows x 9 columns]>


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
testResults = pd.DataFrame(columns=['Treatment','Diagnosis','Prevention','Mechanism','Transmission','Epidemic Forecasting','Case Report'])

for el in df2["label"]:
  rowList = []
  if "Treatment" in el:
    rowList.append(1)
  else:
    rowList.append(0)
  if 'Diagnosis' in el:
    rowList.append(1)
  else:
    rowList.append(0)
  if 'Prevention' in el:
    rowList.append(1)
  else:
    rowList.append(0)
  if 'Mechanism' in el:
    rowList.append(1)
  else:
    rowList.append(0)
  if 'Transmission' in el:
    rowList.append(1)
  else:
    rowList.append(0)
  if 'Epidemic Forecasting' in el:
    rowList.append(1)
  else:
    rowList.append(0)
  if 'Case Report' in el:
    rowList.append(1)
  else:
    rowList.append(0)

  df_length = len(testResults)
  testResults.loc[df_length] = rowList




In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stopwords = stopwords.words('english')
stopwords = set(stopwords)

def tokenize_lemma_stopwords(text):
    text = text.replace("\n", " ")
    # split string into words (tokens)
    tokens = nltk.tokenize.word_tokenize(text.lower())
    # keep strings with only alphabets
    tokens = [t for t in tokens if t.isalpha()]
    # put words into base form
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] 
    tokens = [stemmer.stem(t) for t in tokens]
    # remove short words, they're probably not useful
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    cleanedText = " ".join(tokens)
    return cleanedText

def dataCleaning(df):
    data = df.copy()
    data["content"] =(df["title"]).apply(tokenize_lemma_stopwords)
    return data

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
counterAlpha = 0
counterStopWords =0
cleanedTrainData = dataCleaning(df)
counterAlpha = 0
counterStopWords =0
cleanedTestData = dataCleaning(df2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

for i in range(len(cleanedTrainData["label"])):
  cleanedTrainData["label"][i] = str(cleanedTrainData["label"][i]).split(";")
for i in range(len(cleanedTestData["label"])):
  cleanedTestData["label"][i] = str(cleanedTestData["label"][i]).split(";")

mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(cleanedTrainData["label"])
test_labels = mlb.transform(cleanedTestData["label"])
print(train_labels)
print(test_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[[0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 1]
 [1 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]
[[0 0 0 ... 0 0 1]
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 [0 1 0 ... 0 0 1]]


In [ ]:
torch.cuda.empty_cache()

In [ ]:
import gc
del model
gc.collect()

629

In [ ]:
!nvidia-smi


Fri Jan 14 12:59:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    31W /  70W |  13184MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [13]:
# hyperparameters
MAX_LEN = 20
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 1e-05

In [14]:
from transformers import BertTokenizer, BertModel

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['content']
        self.targets = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [17]:
train_df = cleanedTrainData[:20000]
val_df = cleanedTrainData[20000:]
test_df = cleanedTestData
val_df.reset_index(drop=True, inplace=True)

In [18]:
train_dataset = CustomDataset(train_df,train_labels[:20000], tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df,train_labels[20000:], tokenizer, MAX_LEN)
test_dataset = CustomDataset(test_df,test_labels, tokenizer, MAX_LEN)

In [19]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    drop_last=True
)
test_data_loader = torch.utils.data.DataLoader(test_dataset, 
    batch_size=8,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 7)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

In [22]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [23]:
val_targets=[]
val_outputs=[]

In [24]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if batch_idx%50==0:
            print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      #save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        #save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [25]:
ckpt_path = "/content/drive/MyDrive/datasets/multi-label/curr_ckpt"
best_model_path = "/content/drive/MyDrive/datasets/multi-label/best_model.pt"

In [26]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
Epoch: 1, Training Loss:  0.7173444628715515
Epoch: 1, Training Loss:  0.48879945278167725
Epoch: 1, Training Loss:  0.4460816979408264
Epoch: 1, Training Loss:  0.3951476216316223
Epoch: 1, Training Loss:  0.36039286851882935
Epoch: 1, Training Loss:  0.3265092968940735
Epoch: 1, Training Loss:  0.2973400056362152
Epoch: 1, Training Loss:  0.29337021708488464
Epoch: 1, Training Loss:  0.2888309955596924
Epoch: 1, Training Loss:  0.3922474980354309
Epoch: 1, Training Loss:  0.23836950957775116
Epoch: 1, Training Loss:  0.19783827662467957
Epoch: 1, Training Loss:  0.20539015531539917
Epoch: 1, Training Loss:  0.35085830092430115
Epoch: 1, Training Loss:  0.40098270773887634
Epoch: 1, Training Loss:  0.4686267375946045
Epoch: 1, Training Loss:  0.285885751247406
Epoch: 1, Training Loss:  0.23905763030052185
Epoch: 1, Training Loss:  0.2535453736782074
Epoch: 1, Training Loss:  0.2787538766860962
Epoch: 1, Training Loss:  0.3232952654

In [29]:
def test():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_data_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [31]:
from sklearn import metrics

In [32]:
outputs, targets = test()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.6798780487804879
F1 Score (Micro) = 0.8077698369067253
F1 Score (Macro) = 0.7341940425540406


In [33]:
import numpy as np
#[6:Treatment, 4:Prevention, 3:Mechanism, 1:Diagnosis , 2:Epidemic Forecasting , 5:Transmission , 0:Case Report  ]
predictions = pd.DataFrame(np.nan, index=[i for i in range(6232)],columns=["Case Report","Diagnosis","Epidemic Forecasting","Mechanism","Prevention","Transmission","Treatment"])
#predictions = predictions.append(pd.DataFrame(knnPredictions, columns=predictions.columns))
columns=["Case Report","Diagnosis","Epidemic Forecasting","Mechanism","Prevention","Transmission","Treatment"]
for i in range(7):
  for j in range(6232):
    predictions[columns[i]][j] = outputs[j][i]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [34]:
predictions.replace({False: 0, True: 1}, inplace=True)

In [35]:
predictions = predictions[["Treatment","Diagnosis", "Prevention", "Mechanism", "Transmission","Epidemic Forecasting", "Case Report"]]

In [36]:
predictions.reset_index(drop=True, inplace=True)

In [37]:
predictions.to_csv("/content/predictions.csv")

In [38]:
testResults.to_csv("/content/testResults.csv")

In [39]:
!git clone https://github.com/ncbi/biocreative_litcovid.git

Cloning into 'biocreative_litcovid'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 32 (delta 15), reused 12 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [41]:
!python /content/biocreative_litcovid/biocreative_litcovid_eval.py --gold /content/testResults.csv --pred /content/predictions.csv

validation starts...
validation passes...
label-based measures...
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                      precision    recall  f1-score   support

           Treatment     0.8294    0.7985    0.8137      2204
           Diagnosis     0.8178    0.7178    0.7646      1545
          Prevention     0.8873    0.8857    0.8865      2747
           Mechanism     0.7961    0.7292    0.7612      1071
        Transmission     0.7818    0.3373    0.4712       255
Epidemic Forecasting     0.8389    0.6510    0.7331       192
         Case Report     0.7663    0.6598    0.7090       482

           micro avg     0.8396    0.7782    0.8078      8496
           macro avg     0.8168    0.6828    0.7342      8496
 

In [ ]:
torch.save(model.state_dict(), '/content/model2.pt')

In [ ]:
model = BERTClass()
model.to(device)

In [ ]:
model.load_state_dict(torch.load("/content/model2.pt"))
model.eval()

In [ ]:
from google.colab import files
files.download('/content/predictions.csv') 